# **Proposed CNN Architecture for Second and Third Degree Burn Classification**

This notebook implements the final CNN architecture selected after sequential hyperparameter optimization and subsequent color channel sensitivity analysis.  

Sequential optimization identified a compact three-layer convolutional architecture as the most stable configuration in terms of validation performance. After selecting the optimal hyperparameters, a sensitivity analysis evaluated different color channel combinations (red, green, blue, RG, RB, GB, RGB).  

The green-channel-only configuration achieved the best balance between training and validation performance. Therefore, this notebook reproduces the final model trained exclusively on the green channel.

In [1]:
from keras import layers

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tools import load_images_with_labels, evaluate_model

## Load data

In [ ]:
#Image dimensions
width = 540
height = 960

#Paths where training, validation and test images are stored and organized by class
path = '../data/burn_images/'
path_test = '../data/test_images/'

In [3]:
#Load the images only with the green channel, with their corresponding labels
X, y = load_images_with_labels(path=path, channels='green')

#Split the dataset into 80% training and 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
#Load the intependent test set and their corresponding labels (using only the green channel)
X_test, y_test = load_images_with_labels(path=path, channels='green')

In [5]:
print('Shape of X_train:', X_train.shape)
print('Shape of X_val:', X_val.shape)
print('Shape of X_test:', X_test.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of y_val:', y_val.shape)
print('Shape of y_test:', y_test.shape)

Shape of X_train: (180, 960, 540, 1)
Shape of X_val: (45, 960, 540, 1)
Shape of X_test: (225, 960, 540, 1)
Shape of y_train: (180,)
Shape of y_val: (45,)
Shape of y_test: (225,)


## Model architecture

In [ ]:
cnn = tf.keras.Sequential([
    #Convolutional block 1
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), input_shape=(height, width, 1)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    #Convolutional block 2
    layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2,2)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    #Convolutional block 3
    layers.Conv2D(filters=64, kernel_size=(3,3), strides=(2,2)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),

    #Feature aggregation
    layers.GlobalAveragePooling2D(),

    #Classification stage
    layers.Dense(units=64),
    layers.Activation('relu'),
    layers.Dropout(rate=0.4),
    
    #Output layer: probability of third-degree burn
    layers.Dense(units=1),
    layers.Activation('sigmoid')
])

c:\Users\yamil\miniconda3\envs\ml\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
#Display the final model architecture
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 479, 269, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 479, 269, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 479, 269, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 239, 134, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 119, 66, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 119, 66, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 119, 66, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 59, 33, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 29, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 29, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 29, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,801 (128.13 KB)

 Trainable params: 32,545 (127.13 KB)

 Non-trainable params: 256 (1.00 KB)

## Compilation and training

In [ ]:
#Model compilation
cnn.compile(optimizer='adam', 
            loss='binary_crossentropy', 
            metrics=['accuracy'])

In [ ]:
#Stop training when the model stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, #Wait 10 epochs without improvement before stopping
    restore_best_weights=True #Restore weights from the epoch with the best val_loss
)

#Reduce the learning rate when the model stagnates
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, #Reduce the learning rate by half when there is no improvement
    patience=5, #Wait 5 epochs without improvement before reducing
    min_lr=1e-7 #Minimum allowed learning rate
)

In [ ]:
#Model training
history = cnn.fit(X_train, y_train,
                  epochs=100, batch_size=32,
                  validation_data=(X_val, y_val),
                  callbacks=[early_stopping, reduce_lr])

## Model evaluation

In [ ]:
#Evaluation metrics on the training set
evaluate_model(cnn, X_train, y_train, dataset='Training')

In [ ]:
#Evaluation metrics on the validation set
evaluate_model(cnn, X_val, y_val, dataset='Validation')

In [ ]:
#Evaluation metrics on the independent test set
evaluate_model(cnn, X_test, y_test, dataset='Test')